ref : arXiv:1512.03385v1 [cs.CV] 10 Dec 2015

In [158]:
#import required packages
import tensorflow as tf
from keras.layers import Layer
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
get_ipython().magic('matplotlib inline')

In [159]:
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [160]:
# create the bottleneck for our resnet
def Bottleneck_Identity(X, f, filters, stage, block):
    '''
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    out -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    '''
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    #retrieve filters
    F1, F2, F3 = filters
    
    # save the input value for the shrotcut path and to add it later
    X_shortcut = X
    
    # first component of main path
    out = Conv2D(filters=F1, kernel_size=(1, 1),
              strides=(1, 1), padding='valid', name = conv_name_base + '2a',
              kernel_initializer= glorot_uniform(seed=0))(X)
    out = BatchNormalization(axis=-1, name = bn_name_base + '2a')(out)
    out = Activation('relu')(out)
    
    # second component of the main path
    out = Conv2D(filters=F2, kernel_size=(f, f),
                strides=(1, 1), padding='same', name = conv_name_base + '2b',
                kernel_initializer=glorot_uniform(seed=0))(out)
    out = BatchNormalization(axis=-1, name = bn_name_base + '2b')(out)
    out = Activation('relu')(out)
    
    # third component of the main path
    out = Conv2D(filters= F3, kernel_size=(1, 1),
                strides=(1, 1), padding='valid', name = conv_name_base + '2c',
                kernel_initializer=glorot_uniform(seed=0))(out)
    out = BatchNormalization(axis=-1, name = bn_name_base + '2c')(out)
    
    #upsample X_shortcut to get the same dimensions
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1),
                       strides=(1, 1), padding='valid',
                       kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    
    # Now add shortcut to the main path
    out = Add()([out, X_shortcut])
    # Now pass it through relu activation
    out = Activation('relu')(out)
    
    return out
    

In [161]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [162]:
tf.reset_default_graph()
with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder('float', [1000, 64, 64, 6])
    X = np.random.randn(1000, 64, 64, 6)
    A = Bottleneck_Identity(A_prev, f=3, filters=[2, 4, 8], stage = 1, block='a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev:X, K.learning_phase(): 0})
    print('out = ' + str(out[0][0][0][0]))

out = [0.14369303 0.         1.7858351  0.         0.         0.39647472
 0.         0.03609586]


In [163]:
def Bottleneck_convolutional(X, output_channels, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # save the input for shortcut path
    X_shortcut = X
    
    ### MAIN PATH ###
    # first block
    out = Conv2D(filters=output_channels, kernel_size=(1, 1),
              strides=(2, 2), padding='valid', name = conv_name_base + '2a',
              kernel_initializer=glorot_uniform(seed=0))(X)
    out = BatchNormalization(axis=-1, name = bn_name_base + '2a')(out)
    out = Activation('relu')(out)
    
    # Second block
    out = Conv2D(filters=output_channels, kernel_size=(3, 3),
                strides=(1, 1), padding='same', name = conv_name_base + '2b',
                kernel_initializer=glorot_uniform(seed = 0))(out)
    out = BatchNormalization(axis=-1, name = bn_name_base + '2b')(out)
    out = Activation('relu')(out)
    
    # Third block
    out = Conv2D(filters=output_channels * 4, kernel_size=(1, 1),
                strides=(1, 1), padding='valid', name = conv_name_base + '2c',
                kernel_initializer=glorot_uniform(seed=0))(out)
    out = BatchNormalization(axis=-1, name = bn_name_base + '2c')
    
    # Shortcut path
    X_shortcut = Conv2D(filters=output_channels * 4, kernel_size=(1,1),
                       strides=(2, 2), padding='valid', name = conv_name_base + '1',
                       kernel_initializer= glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=-1, name = bn_name_base + '1')(X_shortcut)
    
    # Now add the shortcut to the output of main path and then pass through relu activation
    out = Add()([X_shortcut, out])
    out = Activation('relu')(out)
    
    return out
    
    

In [164]:
from keras.models import Model

In [165]:
def Resnet50(input_shape=(64, 64, 3), classes = 6):
    """
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    X_input = Input(input_shape) # input tensor
    
    #zero padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    #stage-1
    """
    (7, 7) * 64 => Maxpool(3, 3) : s = 2
    """
    X = Conv2D(filters=64, kernel_size=(7, 7),
                strides=2, name = 'conv1',
                kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=2)(X)
    
    #stage-2
    X = Bottleneck_Identity(X, f=3, filters=[64, 64, 256], stage=2, block='C1')
    X = Bottleneck_Identity(X, f=3, filters=[64, 64, 256], stage=2, block='C2')
    X = Bottleneck_Identity(X, f=3, filters=[64, 64, 256], stage=2, block='C3')
    
    #stage-3
    X = Bottleneck_Identity(X, f=3, filters=[128, 128, 512], stage=3, block='C1')
    X = Bottleneck_Identity(X, f=3, filters=[128, 128, 512], stage=3, block='C2')
    X = Bottleneck_Identity(X, f=3, filters=[128, 128, 512], stage=3, block='C3')
    X = Bottleneck_Identity(X, f=3, filters=[128, 128, 512], stage=3, block='C4')
    
    #stage-4
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C1')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C2')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C3')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C4')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C5')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C6')
    
    #stage-5
    X = Bottleneck_Identity(X, f=3, filters=[512, 512, 2048], stage=5, block='C1')
    X = Bottleneck_Identity(X, f=3, filters=[512, 512, 2048], stage=5, block='C2')
    X = Bottleneck_Identity(X, f=3, filters=[512, 512, 2048], stage=5, block='C3')
    
    X = AveragePooling2D(pool_size=(2, 2), name = 'avg_pool')(X)
    
    #Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name = 'fully_connected' +str(classes),
               kernel_initializer=glorot_uniform(seed=0))(X)
    
    #create Model
    model = Model(inputs=X_input, outputs=X, name = "ResNet50")
    
    return model
    

In [190]:
def Resnet101(input_shape=(64, 64, 3), classes = 6):
    """
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    X_input = Input(input_shape) # input tensor
    
    #zero padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    #stage-1
    """
    (7, 7) * 64 => Maxpool(3, 3) : s = 2
    """
    X = Conv2D(filters=64, kernel_size=(7, 7),
                strides=2, name = 'conv1',
                kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = -1, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=2)(X)
    
    #stage-2
    X = Bottleneck_Identity(X, f=3, filters=[64, 64, 256], stage=2, block='C1')
    X = Bottleneck_Identity(X, f=3, filters=[64, 64, 256], stage=2, block='C2')
    X = Bottleneck_Identity(X, f=3, filters=[64, 64, 256], stage=2, block='C3')
    
    #stage-3
    X = Bottleneck_Identity(X, f=3, filters=[128, 128, 512], stage=3, block='C1')
    X = Bottleneck_Identity(X, f=3, filters=[128, 128, 512], stage=3, block='C2')
    X = Bottleneck_Identity(X, f=3, filters=[128, 128, 512], stage=3, block='C3')
    X = Bottleneck_Identity(X, f=3, filters=[128, 128, 512], stage=3, block='C4')
    
    #stage-4
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C1')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C2')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C3')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C4')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C5')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C6')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C7')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C8')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C9')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C10')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C11')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C12')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C13')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C14')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C15')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C16')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C17')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C18')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C19')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C20')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C21')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C22')
    X = Bottleneck_Identity(X, f=3, filters=[256, 256, 1024], stage=4, block='C23')
    
    #stage-5
    X = Bottleneck_Identity(X, f=3, filters=[512, 512, 2048], stage=5, block='C1')
    X = Bottleneck_Identity(X, f=3, filters=[512, 512, 2048], stage=5, block='C2')
    X = Bottleneck_Identity(X, f=3, filters=[512, 512, 2048], stage=5, block='C3')
    
    X = AveragePooling2D(pool_size=(2, 2), name = 'avg_pool')(X)
    
    #Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name = 'fully_connected' +str(classes),
               kernel_initializer=glorot_uniform(seed=0))(X)
    
    #create Model
    model = Model(inputs=X_input, outputs=X, name = "ResNet101")
    
    return model
    

In [191]:
# Assign the model
model = Resnet101(input_shape=(64, 64, 3), classes=6)
model.summary()

Model: "ResNet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 70, 70, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________

activation_262 (Activation)     (None, 15, 15, 256)  0           add_86[0][0]                     
__________________________________________________________________________________________________
res3C1_branch2a (Conv2D)        (None, 15, 15, 128)  32896       activation_262[0][0]             
__________________________________________________________________________________________________
bn3C1_branch2a (BatchNormalizat (None, 15, 15, 128)  512         res3C1_branch2a[0][0]            
__________________________________________________________________________________________________
activation_263 (Activation)     (None, 15, 15, 128)  0           bn3C1_branch2a[0][0]             
__________________________________________________________________________________________________
res3C1_branch2b (Conv2D)        (None, 15, 15, 128)  147584      activation_263[0][0]             
__________________________________________________________________________________________________
bn3C1_bran

__________________________________________________________________________________________________
res3C4_branch2c (Conv2D)        (None, 15, 15, 512)  66048       activation_273[0][0]             
__________________________________________________________________________________________________
bn3C4_branch2c (BatchNormalizat (None, 15, 15, 512)  2048        res3C4_branch2c[0][0]            
__________________________________________________________________________________________________
conv2d_90 (Conv2D)              (None, 15, 15, 512)  262656      activation_271[0][0]             
__________________________________________________________________________________________________
add_90 (Add)                    (None, 15, 15, 512)  0           bn3C4_branch2c[0][0]             
                                                                 conv2d_90[0][0]                  
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
bn4C4_branch2a (BatchNormalizat (None, 15, 15, 256)  1024        res4C4_branch2a[0][0]            
__________________________________________________________________________________________________
activation_284 (Activation)     (None, 15, 15, 256)  0           bn4C4_branch2a[0][0]             
__________________________________________________________________________________________________
res4C4_branch2b (Conv2D)        (None, 15, 15, 256)  590080      activation_284[0][0]             
__________________________________________________________________________________________________
bn4C4_branch2b (BatchNormalizat (None, 15, 15, 256)  1024        res4C4_branch2b[0][0]            
__________________________________________________________________________________________________
activation_285 (Activation)     (None, 15, 15, 256)  0           bn4C4_branch2b[0][0]             
__________

bn4C7_branch2c (BatchNormalizat (None, 15, 15, 1024) 4096        res4C7_branch2c[0][0]            
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, 15, 15, 1024) 1049600     activation_292[0][0]             
__________________________________________________________________________________________________
add_97 (Add)                    (None, 15, 15, 1024) 0           bn4C7_branch2c[0][0]             
                                                                 conv2d_97[0][0]                  
__________________________________________________________________________________________________
activation_295 (Activation)     (None, 15, 15, 1024) 0           add_97[0][0]                     
__________________________________________________________________________________________________
res4C8_branch2a (Conv2D)        (None, 15, 15, 256)  262400      activation_295[0][0]             
__________

activation_305 (Activation)     (None, 15, 15, 256)  0           bn4C11_branch2a[0][0]            
__________________________________________________________________________________________________
res4C11_branch2b (Conv2D)       (None, 15, 15, 256)  590080      activation_305[0][0]             
__________________________________________________________________________________________________
bn4C11_branch2b (BatchNormaliza (None, 15, 15, 256)  1024        res4C11_branch2b[0][0]           
__________________________________________________________________________________________________
activation_306 (Activation)     (None, 15, 15, 256)  0           bn4C11_branch2b[0][0]            
__________________________________________________________________________________________________
res4C11_branch2c (Conv2D)       (None, 15, 15, 1024) 263168      activation_306[0][0]             
__________________________________________________________________________________________________
bn4C11_bra

__________________________________________________________________________________________________
add_104 (Add)                   (None, 15, 15, 1024) 0           bn4C14_branch2c[0][0]            
                                                                 conv2d_104[0][0]                 
__________________________________________________________________________________________________
activation_316 (Activation)     (None, 15, 15, 1024) 0           add_104[0][0]                    
__________________________________________________________________________________________________
res4C15_branch2a (Conv2D)       (None, 15, 15, 256)  262400      activation_316[0][0]             
__________________________________________________________________________________________________
bn4C15_branch2a (BatchNormaliza (None, 15, 15, 256)  1024        res4C15_branch2a[0][0]           
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
bn4C18_branch2b (BatchNormaliza (None, 15, 15, 256)  1024        res4C18_branch2b[0][0]           
__________________________________________________________________________________________________
activation_327 (Activation)     (None, 15, 15, 256)  0           bn4C18_branch2b[0][0]            
__________________________________________________________________________________________________
res4C18_branch2c (Conv2D)       (None, 15, 15, 1024) 263168      activation_327[0][0]             
__________________________________________________________________________________________________
bn4C18_branch2c (BatchNormaliza (None, 15, 15, 1024) 4096        res4C18_branch2c[0][0]           
__________________________________________________________________________________________________
conv2d_108 (Conv2D)             (None, 15, 15, 1024) 1049600     activation_325[0][0]             
__________

__________________________________________________________________________________________________
activation_337 (Activation)     (None, 15, 15, 1024) 0           add_111[0][0]                    
__________________________________________________________________________________________________
res4C22_branch2a (Conv2D)       (None, 15, 15, 256)  262400      activation_337[0][0]             
__________________________________________________________________________________________________
bn4C22_branch2a (BatchNormaliza (None, 15, 15, 256)  1024        res4C22_branch2a[0][0]           
__________________________________________________________________________________________________
activation_338 (Activation)     (None, 15, 15, 256)  0           bn4C22_branch2a[0][0]            
__________________________________________________________________________________________________
res4C22_branch2b (Conv2D)       (None, 15, 15, 256)  590080      activation_338[0][0]             
__________

activation_348 (Activation)     (None, 15, 15, 512)  0           bn5C2_branch2b[0][0]             
__________________________________________________________________________________________________
res5C2_branch2c (Conv2D)        (None, 15, 15, 2048) 1050624     activation_348[0][0]             
__________________________________________________________________________________________________
bn5C2_branch2c (BatchNormalizat (None, 15, 15, 2048) 8192        res5C2_branch2c[0][0]            
__________________________________________________________________________________________________
conv2d_115 (Conv2D)             (None, 15, 15, 2048) 4196352     activation_346[0][0]             
__________________________________________________________________________________________________
add_115 (Add)                   (None, 15, 15, 2048) 0           bn5C2_branch2c[0][0]             
                                                                 conv2d_115[0][0]                 
__________

In [192]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [193]:
#load our numeric signs dataset
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

In [194]:
# Normalize image vectors into (o, 1) range
X_train = X_train_orig/ 255.0
X_test = X_test_orig/ 255.0

In [195]:
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

In [196]:
print("X_train shape", str(X_train.shape))
print("Y_train shape", str(Y_train.shape))
print("X_test shape", str(X_test.shape))
print("Y_test shape", str(Y_test.shape))

X_train shape (1080, 64, 64, 3)
Y_train shape (1080, 6)
X_test shape (120, 64, 64, 3)
Y_test shape (120, 6)


In [ ]:
model.fit(X_train, Y_train, epochs=2, batch_size=16)

In [176]:
preds = model.evaluate(X_test, Y_test)

120/120 [==============================] - ETA:  - ETA:  - ETA:  - 4s 33ms/step


In [180]:
print(str(preds[0]))
print(str(preds[1]))
print(str(preds))

1.7917092561721801
0.1666666716337204
[1.7917092561721801, 0.1666666716337204]


In [184]:
model.save_weights("ResNet.h5")